In [1]:
# This is a test for converting a single image that contains all of the masks for each
# cell to a series of images containing one mask each.
import os
from PIL import Image
import numpy as np
np.set_printoptions(threshold=np.nan)


ROOT_DIR = os.path.abspath("../../")
im = Image.open(ROOT_DIR + '/test_cell_images/new_images/patched_ids/slice0001_patched_masks_ids.tif')
imarray = np.array(im)



In [5]:
class Mask(object):
    
    ROOT_DIR = os.path.abspath("../../")

    def __init__(self, image_dir, image_name):
        """
        Creates new object Mask. Initializes a 
        image_dir: path after ROOT_DIR to the images
        image_name: file name
        """
        image_dir_path = os.path.join(ROOT_DIR, image_dir)
        self.image_path = os.path.join(image_dir_path, image_name)
        im = Image.open(self.image_path)
        self.mask_array = np.array(im).astype(bool) # mask image as numpy array
        
        self.list_of_masks = [] # contains one np array for each instance (each created from make_mask_array)
        self.visited_points = np.full(self.mask_array.shape, False) # boolean array. True = visited
                
    def visited(self, point):
        # returns the value of visited+points at point
        x = point[0]
        y = point[1]
        return (self.visited_points[x,y])
    
    #def separate_masks(self):
        #Creates each mask instance and appends to self.list_of_masks

    def dfs(self, point, instance_points_array):
        x = point[0] # row
        y = point[1] # col
        if (not self.inBounds(x,y) or self.visited(point)): 
            return None
        self.visited_points[x,y] = True
        if self.mask_array[x,y] == True:
            instance_points_array[x,y] = True

            self.dfs((x+1,y), instance_points_array)
            self.dfs((x,y-1), instance_points_array)
            self.dfs((x-1,y), instance_points_array)
            self.dfs((x,y+1), instance_points_array)

    def inBounds(self, row, col):
        # check if row and col are in bounds
        if row < 0 or row >= self.mask_array.shape[0]:
            return False
        if col < 0 or col >= self.mask_array.shape[1]:
            return False
        else:
            return True

    def make_mask_array(self):
        """
        Returns a mask array containing a single instance.
        mask_tuple_list: list of tuples of where the instance mask is located
        """
        x = 0
        y = 0
        #for (x,y), value in np.ndenumerate(self.mask_array):
        instance_array = np.full(self.mask_array.shape, False)   

        if not self.visited((x,y)):
            self.dfs((x,y), instance_array)
            self.list_of_masks.append(instance_array)
            
            
    
    
    
    

In [6]:
mask1 = Mask('test_cell_images/new_images/patched_ids', 'slice0001_patched_masks_ids.tif')

In [9]:
#print(mask1.mask_array)
#print(mask1.list_of_masks[0][27,21])
from scipy.misc import toimage
toimage(mask1.list_of_masks[0].astype(int)).show()

/Users/justinkim/anaconda3/envs/mask_rcnn/lib/python3.4/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  after removing the cwd from sys.path.


In [7]:
mask1.make_mask_array()

In [8]:


"""
class Mask(object):
    
    ROOT_DIR = os.path.abspath("../../")

    def __init__(self, image_dir, image_name):
        """
        Creates new object Mask. Initializes a 
        image_dir: path after ROOT_DIR to the images
        image_name: file name
        """
        image_dir_path = os.path.join(ROOT_DIR, image_dir)
        self.image_path = os.path.join(image_dir_path, image_name)
        im = Image.open(self.image_path)
        self.mask_array = np.array(im).astype(bool) # mask image as numpy array
        
        self.list_of_masks = [] # contains one np array for each instance (each created from make_mask_array)
        self.visited_points = np.full(self.mask_array.shape, False) # boolean array. True = visited
                
    def visited(self, point):
        # returns the value of visited+points at point
        x = point[0]
        y = point[1]
        return (self.visited_points[x,y])
    
    def merge_visited(self, visited_set):
        # merges self.visited_points with the visited_points_set set from a SingleInstance object
        for point in visited_set:
            x = point[0]
            y = point[1]
            self.visited_points[x,y] = True
    
    def separate_masks(self):
        #Creates each mask instance and appends to self.list_of_masks
        
        class SingleInstance:
            
            def __init__(self, mask_array, starting_point):
                self.mask_array = mask_array #numpy array
                self.starting_point = starting_point # tuple
                self.visited_points_set = set() # list of points we have visited
                self.instance_points = np.full(self.mask_array.shape, False) # True if part of the instance
                
            def dfs(self, point):
                x = point[0] # row
                y = point[1] # col
                if (not self.inBounds(x,y) or point in self.visited_points_set): 
                    return
                self.visited_points_set.add(point)
                if self.mask_array[x,y] == True:
                    self.instance_points[x,y] = True
                    
                    self.dfs((x+1,y))
                    self.dfs((x,y+1))
                    self.dfs((x-1,y))
                    self.dfs((x,y-1))
    
            def inBounds(self, row, col):
                # check if row and col are in bounds
                if row < 0 or row >= self.mask_array.shape[0]:
                    return False
                if col < 0 or col >= self.mask_array.shape[1]:
                    return False
                else:
                    return True
                
            def make_mask_array(self):
                """
                Returns a mask array containing a single instance.
                mask_tuple_list: list of tuples of where the instance mask is located
                """
                mask_array = np.zeros((self.mask_array.shape[0],self.mask_array.shape[1]))
                for point in self.instance_points:
                    x = point[0]
                    y = point[1]
                    mask_array[x,y] = 1
                return mask_array    
                
        for (x,y), value in np.ndenumerate(self.mask_array):
            if not self.visited((x,y)):
                current = SingleInstance(self.mask_array, (x,y))
                current.dfs((x,y))
                self.merge_visited(current.visited_points_set)
                self.list_of_masks.append(current.make_mask_array())
            
            
    
    
    
"""    

IndentationError: unexpected indent (<ipython-input-8-aa0819a9d636>, line 10)

In [ ]:
mask1 = Mask('test_cell_images/new_images/patched_ids', 'slice0001_patched_masks_ids.tif')

In [ ]:
mask1.mask_array

In [ ]:
mask1.separate_masks()

In [ ]:
empty = mask1.make_empty()
empty

In [ ]:
x = []
x.append((3,3))


In [ ]:
x

In [ ]:
x = 0
if x:
    print("false")